In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key= "6FgtlolX4pJWZty8LaPr")

In [3]:
project = rf.workspace("self-sqvzh").project("banana-ripening-process-ttcvo")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Banana-Ripening-Process-1 in yolov8:: 100%|██████████| 15104/15104 [00:01<00:00, 8484.98it/s]


In [4]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 72154, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 72154 (delta 253), reused 204 (delta 196), pack-reused 71813 (from 3)
Receiving objects: 100% (72154/72154), 38.82 MiB | 26.00 MiB/s, done.
Resolving deltas: 100% (54132/54132), done.


In [5]:
%pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00


In [8]:
# Safe diagnostic + low-memory training with automatic fallback
import os, gc, torch, subprocess
from ultralytics import YOLO

# 0. Friendly header
print("=== START: Diagnostics + safe-train snippet ===\n")

# 1. Try to run nvidia-smi (may fail if no driver)
print("Running nvidia-smi (if available):")
try:
    smi = subprocess.run(["nvidia-smi"], capture_output=True, text=True, check=True)
    print(smi.stdout)
except Exception as e:
    print("nvidia-smi not available or failed:", e)

# 2. Safe torch/CUDA diagnostics (avoid memory_summary bug)
print("\nTorch/CUDA diagnostics:")
print("torch version:", torch.__version__)
print("cuda.is_available():", torch.cuda.is_available())
try:
    print("cuda.device_count():", torch.cuda.device_count())
except Exception as e:
    print("Error getting device_count():", e)

if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    try:
        # safer per-device info using lower-level APIs
        for i in range(torch.cuda.device_count()):
            prop = torch.cuda.get_device_properties(i)
            print(f"\nDevice {i}: {prop.name} | total_memory(MB): {prop.total_memory // (1024**2)}")
            # Use basic memory queries instead of memory_summary
            print("  memory_allocated(MB):", torch.cuda.memory_allocated(i) // (1024**2))
            print("  memory_reserved(MB): ", torch.cuda.memory_reserved(i) // (1024**2))
    except Exception as e:
        print("Warning: failed to query some cuda stats:", e)
else:
    print("No CUDA devices visible to torch.")

# 3. Clear caches & set allocator env var to reduce fragmentation
gc.collect()
torch.cuda.empty_cache()
# set small max split to reduce allocation fragmentation (safe to change)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

print("\nCleared caches and set PYTORCH_CUDA_ALLOC_CONF =", os.environ["PYTORCH_CUDA_ALLOC_CONF"])

# 4. Choose device automatically
device = 0 if (torch.cuda.is_available() and torch.cuda.device_count() > 0) else "cpu"
print("\nSelected device for training:", device)

# 5. Low-memory training attempt (very conservative)
data_yaml = "/content/Banana-Ripening-Process-1/data.yaml"  # adapt if different
print("Using data.yaml:", data_yaml)

model = YOLO("yolov8n.pt")   # smallest model

train_kwargs = dict(
    data=data_yaml,
    epochs=5,
    imgsz=256,   # small images to save VRAM
    batch=1,     # batch must be 1 for constrained VRAM
    name="banana_safe_run",
    device=device,
    workers=0
)

# try half precision only if GPU is present
if device != "cpu":
    train_kwargs["half"] = True

print("\nTraining kwargs:", train_kwargs)

try:
    model.train(**train_kwargs)
except Exception as e:
    print("\nTraining raised an exception (showing type and message):", type(e), e)
    print("\nAttempting CPU fallback training (short run) to validate pipeline...")
    try:
        model.train(data=data_yaml, epochs=3, imgsz=320, batch=4, device="cpu", name="banana_cpu_test")
    except Exception as e2:
        print("CPU fallback training also failed:", type(e2), e2)
        print("---- End of snippet. Paste errors here for help. ----")

print("\n=== END: Diagnostics + safe-train snippet ===")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
=== START: Diagnostics + safe-train snippet ===

Running nvidia-smi (if available):
Thu Nov  6 06:41:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|====================

In [9]:
!ls $(dataset.location)

/bin/bash: line 1: dataset.location: command not found
Banana-Ripening-Process-1  sample_data	yolo11n.pt
runs			   ultralytics	yolov8n.pt


In [10]:
# Python way
import os
print("Dataset path:", dataset.location)
print("Files:", os.listdir(dataset.location))


Dataset path: /content/Banana-Ripening-Process-1
Files: ['README.roboflow.txt', 'train', 'data.yaml', 'valid', 'README.dataset.txt', 'test']


In [11]:
!ls runs/detect


banana_safe_run


In [16]:
!ls runs/detect/banana_safe_run/weights


best.pt  last.pt


In [17]:
from ultralytics import YOLO

# adjust path if your run name differs
weights = "runs/detect/banana_safe_run/weights/best.pt"  # or runs/detect/banana_ripening_auto/weights/best.pt
model = YOLO(weights)

# evaluate on validation set and print metrics
metrics = model.val()
print(metrics)


Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 829.5±464.1 MB/s, size: 23.5 KB)
val: Scanning /content/Banana-Ripening-Process-1/valid/labels.cache... 1525 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1525/1525 3.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 96/96 9.2it/s 10.4s
                   all       1525       3012      0.632      0.694      0.702      0.487
             freshripe        264        315      0.632      0.743      0.758      0.593
           freshunripe        185        417      0.682      0.573      0.727      0.505
              overripe        341        438      0.648      0.731      0.784      0.615
                  ripe        424        654      0.627      0.763      0.709      0.494
                rotten       

In [18]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 129.4 MB/s eta 0:00:00


In [20]:
# app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
model = YOLO("runs/detect/banana_safe_run/weights/best.pt")

st.title("🍌 Banana Ripeness Detector")
uploaded = st.file_uploader("Upload an image", type=["jpg","png","jpeg"])
if uploaded:
    img = Image.open(uploaded).convert("RGB")
    res = model.predict(img, conf=0.35)
    out_img = res[0].plot()
    st.image(out_img, caption="Prediction", use_column_width=True)


2025-11-06 07:43:56.621 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.764 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-06 07:43:56.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.767 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.767 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:43:56.769 Thread 'MainThread': mi

In [22]:
# app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image

# Load your trained model
model_path = "runs/detect/banana_safe_run/weights/best.pt"  # change if needed
model = YOLO(model_path)

st.title("🍌 Banana Ripeness Detection App")
st.markdown("Upload an image to check its ripeness stage.")

# File upload
uploaded_file = st.file_uploader("Upload a banana image", type=["jpg", "jpeg", "png"])
if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    with st.spinner("Detecting ripeness..."):
        results = model.predict(image, conf=0.4)
        annotated_img = results[0].plot()  # draw bounding boxes

    st.image(annotated_img, caption="Prediction Result", use_column_width=True)
    st.success("✅ Detection Complete!")


2025-11-06 07:58:53.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.862 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 07:58:53.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [23]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image

# Load your trained YOLO model
MODEL_PATH = "runs/detect/banana_safe_run/weights/best.pt"
model = YOLO(MODEL_PATH)

# Streamlit UI
st.set_page_config(page_title="🍌 Banana Ripeness Detector", layout="centered")
st.title("🍌 Banana Ripeness Detector")
st.markdown("Upload an image to detect its ripeness stage (Unripe / Ripe / Overripe).")

uploaded_file = st.file_uploader("📤 Upload a banana image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    with st.spinner("Detecting ripeness... ⏳"):
        results = model.predict(image, conf=0.35)
        annotated_image = results[0].plot()

    st.image(annotated_image, caption="Predicted Result", use_column_width=True)
    st.success("✅ Detection Complete!")

    # Show detected class counts
    labels = [model.names[int(box.cls)] for box in results[0].boxes]
    if labels:
        st.write("### 🍌 Ripeness Count:")
        counts = {label: labels.count(label) for label in set(labels)}
        for k, v in counts.items():
            st.write(f"**{k}:** {v}")


Writing app.py


In [25]:
# 🚀 Launch Streamlit app from Colab using ngrok tunnel
from pyngrok import ngrok
import subprocess

# Install dependencies (safe even if already installed)
!pip install -q streamlit pyngrok ultralytics pillow

# Open public tunnel
port = 8501
public_url = ngrok.connect(port).public_url
print("🌍 Streamlit App URL:", public_url)

# Start Streamlit
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", str(port)])


ERROR:pyngrok.process.ngrok:t=2025-11-06T08:18:56+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-06T08:18:56+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-06T08:18:56+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [26]:
# Paste this in a notebook cell and run (Colab / remote notebook)
NGROK_AUTH_TOKEN = "3564gD09St1Qg0N3FubFzLVQDLV_4MBYfh1fHhRvYn7cubtmn"   # <-- replace with your token (keep private)

# install libs if needed
!pip install -q pyngrok streamlit

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# optionally verify
print("ngrok authtoken set. Attempting to open tunnel...")
tunnel = ngrok.connect(8501)
print("Public URL:", tunnel.public_url)


ngrok authtoken set. Attempting to open tunnel...
Public URL: https://subcortical-entirely-rashad.ngrok-free.dev


In [27]:
import subprocess
from pyngrok import ngrok

# 1. Define port
port = 8501

# 2. Open public tunnel
public_url = ngrok.connect(port).public_url
print("🌍 Streamlit App URL:", public_url)

# 3. Start Streamlit app (headless mode)
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", str(port), "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)


🌍 Streamlit App URL: https://subcortical-entirely-rashad.ngrok-free.dev
